In [109]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Create a toy tool

In [110]:
from langchain.agents import tool

from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for weather online"""
    return f"Searching for {query}"

print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Search for weather online
{'query': {'title': 'Query', 'description': 'Thing to search for', 'type': 'string'}}


In [111]:
search("foo bar asdf")

'Searching for foo bar asdf'

## Create a real tool

### Weather Search

In [112]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
        'temperature_unit': 'fahrenheit',
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°F'

print(get_current_temperature.name)
print(get_current_temperature.description)
print(get_current_temperature.args)

get_current_temperature
get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.
{'latitude': {'title': 'Latitude', 'description': 'Latitude of the location to fetch weather data for', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}}


In [113]:
from langchain.tools.render import format_tool_to_openai_function
temp_function = format_tool_to_openai_function(get_current_temperature)

In [114]:
get_current_temperature({"latitude": 41.875870, "longitude": -72.801400})

'The current temperature is 36.3°F'

### Wikipedia tool

In [115]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

wikipedia_function = format_tool_to_openai_function(search_wikipedia)

In [116]:
search_wikipedia({"query": "Zach Wilson"})

"Page: Zach Wilson\nSummary: Zachary Kapono Wilson (born August 3, 1999) is an American football quarterback for the New York Jets of the National Football League (NFL). He played college football at BYU, where he was a two-time bowl game MVP, and was selected second overall by the Jets in the 2021 NFL Draft.\n\nPage: 2023 New York Jets season\nSummary: The 2023 season is the New York Jets' 54th season in the National Football League (NFL), their 64th overall, their fifth under general manager Joe Douglas and their third under head coach Robert Saleh. The Jets are looking to improve on their 7–10 record from 2022. After the Seattle Mariners of Major League Baseball (MLB) clinched a playoff berth for the first time since 2001 and the Sacramento Kings of the National Basketball Association (NBA) clinched a playoff berth for first time since 2006, the Jets and the Buffalo Sabres of the National Hockey League (NHL) are now tied for the longest active playoff drought in major North American

In [117]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [118]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "title": "Non-oAuth Scopes example",
    "version": "1.0.0"
  },
  "paths": {
    "/users": {
      "get": {
        "security": [
          {
            "bearerAuth": [
              "read:users",
              "public"
            ]
          }
        ]
      }
    }
  },
  "components": {
    "securitySchemes": {
      "bearerAuth": {
        "type": "http",
        "scheme": "bearer",
        "bearerFormat": "jwt",
        "description": "note: non-oauth scopes are not defined at the securityScheme level"
      }
    }
  }
}
"""

In [119]:
spec = OpenAPISpec.from_text(text) #I haven't been able to get this to work
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

AttributeError: 'super' object has no attribute 'parse_obj'

In [ ]:
model.invoke("what are three pets names")

In [ ]:
model.invoke("tell me about pet with id 42")

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [120]:
from langchain.chat_models import ChatOpenAI
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [121]:
# Prompt for weather it will use the weather function
model.invoke("what is the weather in SF right now?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

In [122]:
# Prompt for fact it will use the wikipedia function
model.invoke("what is langchain")


AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{\n  "query": "langchain"\n}'}})

In [123]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [124]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

In [142]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
chain = prompt | model | OpenAIFunctionsAgentOutputParser()
weather_result = chain.invoke({"input": "what is the weather in sf right now"})
print(type(weather_result))
wiki_result = chain.invoke({"input": "what is langchain"})
print(type(wiki_result))
regular_result = chain.invoke({"input": "tell me a joke"})
print(type(regular_result))

<class 'langchain.schema.agent.AgentActionMessageLog'>
<class 'langchain.schema.agent.AgentActionMessageLog'>
<class 'langchain.schema.agent.AgentFinish'>


In [137]:
weather_result.tool

'get_current_temperature'

In [138]:
weather_result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [139]:
get_current_temperature(weather_result.tool_input)

'The current temperature is 63.2°F'

In [141]:
search_wikipedia(wiki_result.tool_input)

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Prompt engineering\nSummary: Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model. A prompt is natural language text describing the task that an AI should perform.A prompt for a text-to-text model can be a query such as "what is Fermat\'s little theorem?", a command such as "write a poem about leaves falling", a short statement of feedback (for example, "too verbose", "too formal", "rephrase again", "omit this word") or a longer statement including context, instructions, and input data. Prompt engineering may involve phrasing a query, specifying a style, providing relevant context 

In [145]:
regular_result.return_values

{'output': "Sure, here's a joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"}

In [146]:
# this seems like a hack.  Why can't this be polymorphic?

from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [147]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [149]:
chain.invoke({"input": "What is the weather in san francisco right now?"})

'The current temperature is 63.2°F'

In [150]:
chain.invoke({"input": "What is langchain?"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Prompt engineering\nSummary: Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model. A prompt is natural language text describing the task that an AI should perform.A prompt for a text-to-text model can be a query such as "what is Fermat\'s little theorem?", a command such as "write a poem about leaves falling", a short statement of feedback (for example, "too verbose", "too formal", "rephrase again", "omit this word") or a longer statement including context, instructions, and input data. Prompt engineering may involve phrasing a query, specifying a style, providing relevant context 

In [151]:
chain.invoke({"input": "tell me a joke!"})

"Sure, here's a joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"